In [61]:
%reload_ext autoreload

%autoreload 2
import numpy as np
from models import baseline_mlp
import torch
from sklearn.preprocessing import StandardScaler


In [62]:
# FILES TO CHANGE

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/harvey/kfold/'

In [63]:
def run_experiment(num_hidden, learning_rate, data_dir):
    
    # load data from files
    X_labeled_train_glove = np.load(data_dir + 'X_labeled_train.npy')
    X_histories_train_glove = np.load(data_dir + 'X_histories_train.npy')
    X_labeled_test_glove = np.load(data_dir + 'X_labeled_test.npy')
    X_histories_test_glove = np.load(data_dir + 'X_histories_test.npy')
    
    y_train = torch.Tensor(np.load(data_dir + 'y_train.npy'))
    y_test = torch.Tensor(np.load(data_dir + 'y_test.npy'))
    
    X_labeled_train_tfidf = np.load(data_dir + 'trainTweets.npy')
    X_histories_train_tfidf = np.load(data_dir + 'trainHistories.npy')
    X_labeled_test_tfidf = np.load(data_dir + 'testTweets.npy')
    X_histories_test_tfidf = np.load(data_dir + 'testHistories.npy')
    
    scaler1 = StandardScaler()
    scaler2 = StandardScaler()
    scaler3 = StandardScaler()
    scaler4 = StandardScaler()
    
    # TODO FIX NAMES!           
    X_labeled_train_glove_norm    = scaler1.fit_transform(X_labeled_train_glove)
    X_histories_train_glove_norm  = scaler2.fit_transform(X_histories_train_glove)
    X_labeled_test_glove_norm     = scaler1.transform(X_labeled_test_glove)
    X_histories_test_glove_norm  = scaler2.transform(X_histories_test_glove)
    
    X_labeled_train_tfidf_norm    = scaler3.fit_transform(X_labeled_train_tfidf)
    X_histories_train_tfidf_norm  = scaler4.fit_transform(X_histories_train_tfidf)
    X_labeled_test_tfidf_norm     = scaler3.transform(X_labeled_test_tfidf)
    X_histories_test_tfidf_norm   = scaler4.transform(X_histories_test_tfidf)
    
    # merge data
    X_train = torch.Tensor(np.concatenate((X_labeled_train_glove_norm, X_labeled_train_tfidf_norm,
                              X_histories_train_glove_norm, X_histories_train_tfidf_norm),
                             axis = 1))
    X_test = torch.Tensor(np.concatenate((X_labeled_test_glove_norm, X_labeled_test_tfidf_norm,
                             X_histories_test_glove_norm, X_histories_test_tfidf_norm),
                            axis = 1))

    # build and train model
    
    model = baseline_mlp.MLP(input_size=800, hidden_dim=num_hidden, learning_rate=learning_rate)
    
    model.learn(X_train, y_train, epochs = 3)
    
    # evaluate
    return model.get_accuracy(X_test, y_test)

def cross_validate(num_hidden, learning_rate, data_dir, folds = 10):
    accuracy_sum = 0.0
    for i in range(folds):
        fold_dir = data_dir + str(i) + '/'
        print('running experiment', i)
        acc = run_experiment(num_hidden,learning_rate,fold_dir)
        print('result', acc)
        accuracy_sum += acc
        
    return accuracy_sum / folds


In [64]:
# cross validate search for hyper-parameters
learning_rates = [0.1, 0.07, 0.05, 0.03, 0.01, 0.007, 0.005,
                  0.003, 0.001, 0.0007, 0.0005, 0.0003, 0.0001]
hidden_dims = [100,300,500,700,900,1100,1300,1500]
results = np.zeros((len(learning_rates), len(hidden_dims)))
tests = 1
for i, l in enumerate(learning_rates):
    for j, h in enumerate(hidden_dims):
        print('starting test', tests,'hidden', h, 'learning', l)
        results[i,j] = cross_validate(h, l, NPY_INPUT_DIR) # TODO CHECK THESE PARAMS ARE RIGHT
        tests += 1
        print('test outcome', results[i,j])
        print('************************************************************')
        
np.save('results.npy', results) # to save results
print(results)


starting test 1 hidden 100 learning 0.1
running experiment 0
epoch: 0 learning rate: [0.1]
[1,   200] loss: 6.110
[1,   400] loss: 6.631
[1,   600] loss: 6.942
epoch: 1 learning rate: [0.05]
[2,   200] loss: 7.046
[2,   400] loss: 6.770
[2,   600] loss: 6.666
epoch: 2 learning rate: [0.025]
[3,   200] loss: 6.355
[3,   400] loss: 7.115
[3,   600] loss: 6.942
result 0.759375
running experiment 1
epoch: 0 learning rate: [0.1]
[1,   200] loss: 6.322
[1,   400] loss: 6.597
[1,   600] loss: 6.321
epoch: 1 learning rate: [0.05]
[2,   200] loss: 6.286
[2,   400] loss: 7.322
[2,   600] loss: 6.113
epoch: 2 learning rate: [0.025]
[3,   200] loss: 6.631
[3,   400] loss: 6.528
[3,   600] loss: 6.804
result 0.740625
running experiment 2
epoch: 0 learning rate: [0.1]


KeyboardInterrupt: 